In [ ]:
from bs4 import BeautifulSoup
import requests
import html5lib
import pandas as pd

In [ ]:
# Declaring the empty lists to collect values from individual properties
Address = []
Type_of_Property = []
Rent = []
Deposit =[]
Beds = []
Baths = []
Landlord_Name = []
Landlord_Contact = []

In [ ]:
for a in range(30):
    
    baseurl = 'https://www.gosection8.com/Tenant/tn_Results.aspx?Address=Dallas%20County,%20TX&minRent=0&maxRent=20000&propertyTypeList=All%20Property%20Type&bedrooms=0&bathrooms=1&Accessible=False&radius=&pictures=False&pets=False&ac=False&coveredParking=False&MaxSqFt=20000&MinSqFt=0&keyword=&AgeRestricted=False&smoking=False&pg='+str(a)
    # downloading the html of the webpage
    r = requests.get(baseurl)
    # using beautiful soup to parse through elements of the downloaded html tree
    soup = BeautifulSoup(r.content, 'html5lib')
    # iterationcount gives the count of all the properites listed on a single page
    iterationcount = len(soup.find_all('div', attrs={"class":"listing-content"}))
    
    # Scrapping the URLs of all the individual housing listed on a single page
    l = list()
    for i in range(iterationcount):
        loop_id= "MainContentPlaceHolder_properties_pnlDiv_"+str(i)
        l.append(soup.find('div', attrs={"id":loop_id})['onclick'].strip().split("'")[1])
    # Scrapping the content from each property
    for element in l:
        linkurl = 'https://www.gosection8.com'+element
        r_sub = requests.get(linkurl)
        soup_sub = BeautifulSoup(r_sub.content, 'html5lib')

        Type_of_Property.append(soup_sub.find("span", id="MainContentPlaceHolder_ptype").text)
        Rent.append(soup_sub.find("span", id="MainContentPlaceHolder_rdep").text)
        Deposit.append(soup_sub.find("span", id="MainContentPlaceHolder_rdep2").text)
        Beds.append(soup_sub.find("span", id="MainContentPlaceHolder_bb").text)
        Baths.append(soup_sub.find("span", id="MainContentPlaceHolder_bb2").text)
        
        if soup_sub.find("span", id="MainContentPlaceHolder_Contact1_landlordName") == None:
            Landlord_Name.append(soup_sub.find("span", id="MainContentPlaceHolder_LLNameLbl").text)
        else: 
            Landlord_Name.append(soup_sub.find("span", id="MainContentPlaceHolder_Contact1_landlordName").text)
        
        if soup_sub.find("span", id="MainContentPlaceHolder_Contact1_landlordPhone") == None:
            Landlord_Contact.append(soup_sub.find("span", id="MainContentPlaceHolder_lblPhoneDisplay").text)
        else: 
            Landlord_Contact.append(soup_sub.find("span", id="MainContentPlaceHolder_Contact1_landlordPhone").text)
            
        #Landlord_Contact.append(soup_sub.find("span", id="MainContentPlaceHolder_Contact1_landlordPhone").text)
        Address.append(soup_sub.find("span", id="addr").text+', '+ soup_sub.find("span", id="addr2").text)

In [ ]:
# Creating dataframe
df = pd.DataFrame(columns = ["Address","Type_of_Property","Landlord_Name","Phone_Number","Beds","Baths","Rent","Deposit"])
df["Address"] = Address
df["Type_of_Property"] = Type_of_Property
df["Rent"] = Rent
df["Deposit"] = Deposit
df["Beds"] = Beds
df["Baths"] = Baths
df["Landlord_Name"] = Landlord_Name
df["Phone_Number"] = Landlord_Contact

In [ ]:
# Converting dataframe to csv
df.to_csv(r'C:\Users\prava\Desktop\UTD_SM\Fall 19\CPAL_Project\section8Scraped.csv',index=None)